In [1]:
import random 
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
import json
import csv
from tqdm import tqdm
import re
from datetime import datetime

In [3]:
page= 1
base_url = "https://udn.com/api/more"

news_list = []

HEADERS = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
}    
channelId = 2
type_ = 'searchword'
search_ = '%E2%80%BB%20%E6%8F%90%E9%86%92%E6%82%A8%EF%BC%9A%E7%A6%81%E6%AD%A2'
query = f"page={page+1}&id=search:{search_}&channelId={channelId}&type={type_}"
news_list_url = base_url + '?' + query

print(news_list_url)

https://udn.com/api/more?page=2&id=search:%E2%80%BB%20%E6%8F%90%E9%86%92%E6%82%A8%EF%BC%9A%E7%A6%81%E6%AD%A2&channelId=2&type=searchword


In [4]:
def get_news_list(page_num):
    ### total:289 pages
    ### 參考資料：https://blog.jiatool.com/posts/udn_spider/
    
    """
    爬取新聞列表
    page_num:網頁頁數
    """
    base_url = "https://udn.com/api/more"

    news_list = []
    
    HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
    }
    
    for page in tqdm(range(page_num)):
        ### 找出 Ajax 去請求的網址，然後照他的請求方式、參數來取得資料。
        
        channelId = 2
        type_ = 'searchword'
        search_ = '%E2%80%BB%20%E6%8F%90%E9%86%92%E6%82%A8%EF%BC%9A%E7%A6%81%E6%AD%A2'
        query = f"page={page+1}&id=search:{search_}&channelId={channelId}&type={type_}"
        news_list_url = base_url + '?' + query
        
        r = requests.get(news_list_url, headers=HEADERS)
        news_data = r.json()
        news_list.extend(news_data['lists'])
        time.sleep(random.uniform(1, 2))
    
    news_udn = [] ###網址是udn.com，主要抓這邊的資料
    news_other = [] ###網址是其他的，可能是其他新聞網或是udn附屬的其他網站，資料格式不太一樣
    
    for i in news_list:
        json_ = {
                'title':i['title'].replace('\u2003',''),
                'link':i['titleLink'],
                'cateLink':i['cateLink'],
                'cateTitle':i['cateTitle'],
                'time':i['time']['date'],
            }
        if i['titleLink'].split('/')[2] == 'udn.com':
            news_udn.append(json_)
        else:
            news_other.append(json_)
            
    ### 先寫進.txt，以後收集資料可以抓取來用        
    with open('udn_com.txt', 'w') as outfile:
        json.dump(news_udn, outfile)  
    
    with open('udn_other.txt', 'w') as outfile:
        json.dump(news_other, outfile)
        
    return news_udn

def get_news_home_content(url):

    """
    爬取網頁內文
    url:網址
    """
    
    HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
    }
    r = requests.get(url, headers=HEADERS)
    sp=BeautifulSoup(r.text,'lxml')

    try:
        author=sp.find('span','article-content__author').text
        Author = (re.sub(r'[\n ]*', '', author))
    except AttributeError:
        Author = 'None'
    
    try:
        ad = ".innity-apps-underlay-ad {z-index: 34 !important; }.innity-apps-underlay-ad ~ .header {z-index: 35;}.innity-apps-underlay-ad ~ .main-content .inline-ads { background: transparent;}.article-content__editor video {max-width: 100%;} /* to be remove*/    googletag.cmd.push(function() { googletag.display('ads-inline'); });"
        content = sp.find('section','article-content__editor').text
        Content = (re.sub(r'[\n\r]*', '', content).replace(ad, ''))
    except AttributeError:
        Content = 'None'
    json_ = {
        
        'author' : Author,
        'content' : Content
        
    }
    return json_

In [6]:
all_list = get_news_list(289)

100%|██████████| 289/289 [11:42<00:00,  2.43s/it]


In [47]:
len(all_list)

3100

### 去除掉標題可能是非酒類相關文章

In [66]:
news_alcohol = []
news_accident = []

for i in tqdm(range(len(all_list))):
    if '酒駕' in all_list[i]['title']:
        news_accident.append(all_list[i])
    elif '酒測' in all_list[i]['title']:
        news_accident.append(all_list[i])
    elif '酒瘋' in all_list[i]['title']:
        news_accident.append(all_list[i]) 
    else:
        news_alcohol.append(all_list[i])
        

100%|██████████| 3100/3100 [00:00<00:00, 375757.66it/s]


In [67]:
len(news_accident)

312

In [68]:
len(news_alcohol)

2788

### 爬取內文

In [71]:
author_content = []
for i in tqdm(range(len(news_alcohol))):
    data = get_news_home_content(news_alcohol[i]['link'])
    author_content.append(data)

100%|██████████| 2788/2788 [1:10:18<00:00,  1.51s/it]


###  整理資料

In [73]:
for i in tqdm(range(len(author_content))):
    news_alcohol[i]['author'] = author_content[i]['author']
    news_alcohol[i]['content'] = author_content[i]['content']

100%|██████████| 2788/2788 [00:00<00:00, 226066.07it/s]


### 輸出.json

In [75]:
with open('0213udn_alcohol.json','a') as outfile:
    json.dump(news_alcohol, outfile, ensure_ascii=False)
    outfile.write('\n')

### 匯入mongoDB

In [76]:
from pymongo import MongoClient

In [77]:
client = MongoClient('mongodb://localhost:27017/')
client_db = client['simi']
client_col = client_db['udn_酒類新聞']

with open('0213udn_alcohol.json') as f:
    file_data = json.load(f)

client_col.insert_many(file_data)